<center><strong>Sistem Temu Kembali Informasi</strong><br />
<strong><font color="blue">Semester Gasal T.A. 2020/2021</font></strong><br />
</center>

<strong>Outline pertemuan minggu ke-4</strong><br />
<li> Word Embedding </li>

### Word Embedding

<p> Representasi dari teks ke dalam dense matriks. Term yang memiliki makna serupa berada pada jarak yang berdekatan pada ruang vektor </p>
<p> VSM merepresentasikan teks ke dalam sparse matriks </p>
<p> Sparse matriks memiliki ukuran yang besar dan memiliki banyak nilai 0 </p>
<p> Dense matriks ukurannya lebih kecil, dapat diinisialisasi, dan tidak memiliki nilai 0 </p>
<h2><img alt="" src="figures/3_word_embeddings.png" style="height: 296px ; width: 602px" /></h2>

#### Algoritma word embedding
<ul>
    <li> Skip-gram </li>
    <li> CBoW (Continuous Bag of Words)</li>
    <li> GloVe (Global Vector)</li>
    <li> FastText </li>
    <li> BERT (based on context) </li>
</ul>

#### Skip Gram dan CBoW
<h2><img alt="" src="figures/sg-cbow.png"/></h2>

<p> Dimana w(t) merupakan kata target dan w(t-n) - w(t+n) adalah konteks (kata yang berada disekitar kata target. n merupakan ukuran dari window size. Apabila n = 2 maka konteks didefinisikan sebagai 2 kata sebelum dan setelah kata target.</p>
<p> Contoh: "the big brown bear and the fox"
    <br>Apabila kata target adalah = "brown" dan n = 2 
    <br>Maka: 
    <br>w(t) = "brown"
    <br>w(t-2) = "the"
    <br>w(t-1) = "big"
    <br>w(t+1) = "bear"
    <br>w(t+2) = "and"
</p>
    

In [2]:
# download 3 categoies from 20newgroup dataset
# save it as binary

import os
import pickle
from sklearn.datasets import fetch_20newsgroups

categories = ['sci.med', 'talk.politics.misc',  'rec.autos']
data = fetch_20newsgroups(subset='train', categories=categories,remove=('headers', 'footers', 'quotes'))

dst_name = "20newsgroup.pckl"
dst_path = os.path.join("data", dst_name)
with open(dst_path, 'wb') as fout:
    pickle.dump(data, fout)

In [3]:
# loada data pickle
with open(dst_path, 'rb') as fin:
    data = pickle.load(fin)

data = [doc for doc in data.data]

In [4]:
data[6]

'\nOk boys and girls,\n\n"What was the \'Ogadan War\'????"\n\nThe Money Raised in Band-Aid covered How Much of\nthe Cost of Which Soviet Client State to replace what\ncatagory of weapon system lost in the aforementioned war?\n\nWhy was the Joke: "We arm the World." Really Not that funny?\n\nGonzo Station is the designation for WHICH USN Op Area?\nand the primary threat targets in the Area Were:.....\n\nciao\ndrieux\n\n\n'

In [5]:
# preprocessing
import re
from nltk import sent_tokenize
from nltk import word_tokenize

def preprocess(doc):
    sents = sent_tokenize(doc)
    sents_tok = list() # tokenisasi kalimat
    for s in sents:
        s = s.strip().lower() # case folding dan menghilangkan new line
        s = s.replace("\n", " ") # menggantikan \n dengan spasi
        s = re.sub(r'[^a-zA-Z0-9 ]', ' ', s) # menghapus simbol
        s = re.sub(' +', ' ', s) # menghapus repetitive space
        
        sents_tok.append(s)
    
    return " ".join(sents_tok)

docs = list()
for d in data:
    docs.append(preprocess(d))
#docs

In [6]:
# Untuk membentuk word embedding digunakan library Gensim
# Format inputan dari Gensim adalah list of words untuk setiap kalimat
# Contoh pada dokumen:
#       Tiger is the biggest cat alive. The tigers species can be divided into 5 groups.
# Inputan yang dibutuhkan Gensim adalah:
#       ["tiger", "is", "the", "biggest", "cat", "alive"], ["the", "tigers", "species", 
#        "can", "be", "devided", "into", "5", "groups"]
#
doc_gensim = [word_tokenize(d) for d in docs]
#print(doc_gensim)

In [7]:
# train skip-gram dengan library gensim
#
from gensim.models import Word2Vec

d = 300
model_sg = Word2Vec(doc_gensim, min_count=1, size=d, window=5, workers=-1,)
print("DONE")

C:\Users\Rizka\Anaconda3\envs\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


DONE


In [8]:
import os

model_path = os.path.join('model', 'model_sg.model')
# model dari word embedding dapat disimpan dengan fungsi di bawah ini
model_sg.save(model_path)

# untuk menggunakan model yang telah disimpan sebelumnya dapat dilakukan
# dengan memanfaatkan fungsi load, beberapa model membutuhkan waktu yang lama
# karena memiliki ukuran yang besar
model_sg_load = Word2Vec.load(model_path)

print('DONE')


DONE


#### Word2 Vec menggunakan matriks dense

<p>Penggunaan memori oleh Gensim</p>
<p>Jumlah kata x size x 12 bytes <br>
    Contoh: <br>
    Jika terdapat 100.000 kata unik dengan menggunakan dimensi embedding 200 <br>
    Maka memori yang dibutuhkan = 100.000 x 200 x 12 bytes = ~229MB
</p>

In [9]:
# Melihat vektor suatu kata
model_sg.wv['car']

array([ 5.60016197e-04, -7.80494651e-04,  8.53844249e-05,  5.06553897e-06,
        6.77944685e-04, -1.50504883e-03, -2.22184652e-04,  1.31922297e-03,
        1.55192008e-03,  1.17945485e-03, -6.71928399e-04, -1.32007641e-04,
        1.97516536e-04,  5.52323938e-04, -1.15502952e-03, -7.38360512e-04,
       -1.40060426e-03,  1.04278733e-03,  5.50589466e-04,  7.50768872e-04,
       -2.12674597e-04,  1.12611393e-03, -7.43451179e-04, -7.87006866e-05,
       -3.39416030e-04, -1.35552240e-04, -9.09262890e-05,  1.57095736e-03,
        4.25584498e-04, -9.79802455e-04,  1.45992637e-03,  1.18217918e-04,
        1.46167702e-04,  1.43519649e-03,  1.15383952e-03,  6.28191163e-04,
        5.27482189e-04,  7.87348486e-04,  8.05151125e-04,  1.25717488e-03,
       -1.54108042e-03,  1.92914275e-04,  1.18085439e-03, -5.00844981e-05,
        7.21530014e-05, -1.02488894e-03, -1.63354212e-03,  8.90475872e-04,
       -1.09124882e-03,  6.49984344e-04,  1.26502695e-04, -1.31450966e-03,
        8.30539444e-04, -

In [10]:
# Menghitung similarity vektor antar kata

model_sg.wv.similarity('car', 'car')

C:\Users\Rizka\Anaconda3\envs\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


1.0

In [23]:
#  Menampilkan top-N similar words

model_sg.wv.similar_by_word('car', topn=10)

C:\Users\Rizka\Anaconda3\envs\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('controversy', 0.2052428126335144),
 ('line', 0.20026910305023193),
 ('sox', 0.19885006546974182),
 ('textbook', 0.19850188493728638),
 ('amc', 0.19668230414390564),
 ('doses', 0.19454580545425415),
 ('heard', 0.19357556104660034),
 ('tons', 0.19148902595043182),
 ('panther', 0.1892298460006714),
 ('advisers', 0.18916377425193787)]

In [22]:
# Mencari yang paling mirip dengan kata 'cars'
model_sg.wv.most_similar('cars')

C:\Users\Rizka\Anaconda3\envs\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('directing', 0.21145768463611603),
 ('trigger', 0.20305946469306946),
 ('implementing', 0.20233061909675598),
 ('miazawa', 0.19904084503650665),
 ('19104', 0.19754955172538757),
 ('rx', 0.19124704599380493),
 ('72', 0.19081518054008484),
 ('ununsual', 0.18937884271144867),
 ('doubts', 0.18880492448806763),
 ('chit', 0.18676556646823883)]

<p><img alt="" src="figures/3_cosine.png" style="height:400px; width:683px" /></p>

In [16]:
# error jika kata tidak ada di training data

kata = 'entir'
try:
    print(model_sg.wv.most_similar(kata))
except:
    print('error! kata "',kata,'" tidak ada di training data')

error! kata " entir " tidak ada di training data


### Note: ###
<ul>
    <li> Pada word2vec apabila suatu kata tidak terdapat dalam list vocabulary maka akan error </li>
    <li> Kesalahan pengetikan (typo) bisa meninmbulkan error </li>
    <li> Untuk mangatasi hal ini dapat dilakukan dengan menggunakan algoritma lain, seperti FastText </li>

In [19]:
# "predict" vector for new data without re-training from the beginning
d1 = ['new','generation','nvidia','gpu','is','rtx']
d2 = ['deep','learning','computation','mostly', 'on', 'gpu']
d3 = ['the','rtx','gpu','capable','super','sampling','ssdl']
D = [d1,d2,d3]
model_sg.train(D, total_examples=len(D), epochs=model_sg.epochs)
print(model_sg.predict_output_word('gpu'))

[('intrigues', 4.4517652e-05), ('enduring', 4.4517652e-05), ('cardinal', 4.4517652e-05), ('shirer', 4.4517652e-05), ('chancellors', 4.4517652e-05), ('undermining', 4.4517652e-05), ('waning', 4.4517652e-05), ('strasser', 4.4517652e-05), ('schleicher', 4.4517652e-05), ('unite', 4.4517652e-05)]


<h3 id="Latihan:"><font color="blue">Latihan 1:</font></h3>

<ul> 
    <li> Bangunlah word embedding dengan data 3 kategori lainnya dari data 20newsGroup</li>
    <li> Lakukan training menggunakan Skip-Gram dan CBoW </li>
    <li> Apakah vektor yang dihasilkan oleh suatu kata sama? ketika menggunakan Skip-Gram dan CBoW </li>
    <li> Apakah hasil top-n similar memperoleh vocabulary yang sama? </li>
    <li> Apakah jumlah dimensi berpengaruh pada hasil top-n most similar? </li>

In [ ]:
# Kerjakan latihan 1 pada cell berikut ini


## FastText (Facebook-2016)##
<ul>
    <li> Menggunakan Sub-words: app, ppl, ple - apple, sehingga dapat mengatasi permasalahan typo atau tidak adanya suatu kata dalam vocabularies </li>
    <li> Paper: https://arxiv.org/abs/1607.04606  </li>
    <li> Website: https://fasttext.cc/ </li>
    <li> Source: https://github.com/facebookresearch/fastText </li>
</ul>

In [20]:
from gensim.models import FastText

dim = 300 # Jumlah neurons = ukuran vektor = jumlah kolom
model_ft = FastText(doc_gensim, size=dim, window=5, min_count=2, workers=-1)
'Done'

'Done'

In [24]:
#  Menampilkan top-N similar words

model_ft.wv.similar_by_word('car', topn=10)

C:\Users\Rizka\Anaconda3\envs\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('carb', 0.41024261713027954),
 ('carbon', 0.38731497526168823),
 ('carbs', 0.38285455107688904),
 ('care', 0.37935227155685425),
 ('carl', 0.3629686236381531),
 ('card', 0.3505946099758148),
 ('cars', 0.34071898460388184),
 ('carlos', 0.3346530497074127),
 ('career', 0.3300458788871765),
 ('carter', 0.31315213441848755)]

In [21]:
# Mencari yang paling mirip dengan kata 'cars'
model_ft.wv.most_similar('cars')

C:\Users\Rizka\Anaconda3\envs\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('carbs', 0.3426836133003235),
 ('car', 0.34071898460388184),
 ('cardiac', 0.3207300901412964),
 ('card', 0.3046035170555115),
 ('carb', 0.3021414875984192),
 ('care', 0.2941848039627075),
 ('carried', 0.2819857895374298),
 ('carry', 0.27728378772735596),
 ('carriers', 0.27277398109436035),
 ('carlos', 0.2711375057697296)]

In [27]:
# Melihat similarity antar kata

print(model_ft.wv.similarity('cars', 'car'))
print(model_sg.wv.similarity('cars', 'car'))

0.34071898
0.013699394


C:\Users\Rizka\Anaconda3\envs\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [28]:
# Tidak error jika kata tidak ada di training data

kata = 'beckmans'
try:
    print(model_ft.wv.most_similar(kata))
except:
    print('error! kata "',kata,'" tidak ada di training data')
    

# Tidak terjadi error saat kata tidak terdapat pada vocabulary

[('germans', 0.36002957820892334), ('humans', 0.3300623595714569), ('plans', 0.2815879285335541), ('lemans', 0.27253299951553345), ('canadians', 0.2638093829154968), ('organs', 0.263003408908844), ('fans', 0.25659996271133423), ('means', 0.23399443924427032), ('cans', 0.2265084683895111), ('becoming', 0.2197355329990387)]


C:\Users\Rizka\Anaconda3\envs\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


<h3 id="Latihan:"><font color="blue">Latihan 2:</font></h3>

<ul>
	<li>Apakah kelebihan dan kekurangan WE secara umum?</li>
	<li>Apakah kira-kira aplikasi yang dapat memanfaatkan WE?</li>
	<li>Apakah bisa dijadikan representasi dokumen? Bagaimana caranya?</li>
	<li>Bergantung pada apa sajakah performa model WE?</li>
</ul>

* Preprocessing apa yang sebaiknya dilakukan pada model Word Embedding?
* Apakah Pos Tag bermanfaat disini? Jika iya bagaimana menggunakannya?

In [ ]:
# Kerjakan latihan 2 pada cell berikut ini
